reference https://www.blakeporterneuro.com/learning-python-project-3-scrapping-data-from-steams-community-market/
this is for scrapping market place of a particular game which is technacally not what I want as I am interested more in the payer count

video that helped me espicially in the inital scrapping of review to understanad the workflow of scrapping and acessing data https://www.youtube.com/watch?v=Id2iYV3EfG4

# 1. Web Scrapping

a) get_app_id





In [71]:

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time
import random
import requests
import json

# automatically laveling AUTOMATICALLY LAELING A THREE DATA SET

In [72]:
# b) review scrapping




# https://www.geeksforgeeks.org/response-json-python-requests/
def get_app_id():
    
    req = requests.get("https://api.steampowered.com/ISteamApps/GetAppList/v2")
    
    full_id_json = req.json()
    
    data = full_id_json["applist"]["apps"]
    
    print(data)
    
    app_id = []
    name = []
    
    
    # A little bit of bad code, no try except
    for i in data:
        
        
        app_id_instance = i['appid']
        name_instance = i['name']
        
        
        # some names are empty
        if not name_instance :
           continue
        
        app_id.append(app_id_instance)
        name.append(name_instance)
        

    
    return zip(app_id,name)


#  https://www.geeksforgeeks.org/create-pandas-dataframe-from-lists-using-zip/
app_data = get_app_id()

df = pd.DataFrame(app_data, columns=['game_id', 'name'])


df.to_csv("steam_apps.csv", index=False)

print(df.head())


df_sample = df.sample(n=100)

# Save the sample to a new CSV file (optional)
df_sample.to_csv("steam_apps_sample.csv", index=False)

# Print the first few rows of the sample
print(df_sample.head())

In [73]:
# each steam page has a unqiue game id in their url so this could help 

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
import time
import random

options = Options()


# game id
game_id = 413150 

# 413150  Stardew Valley
# 1623730 PalWorld

#url_template
game_url = f"https://steamcommunity.com/app/{game_id}/reviews/?p=1&browsefilter=trendyear&filterLanguage=english"
print(game_url)


# https://www.selenium.dev/selenium/docs/api/rb/Selenium/WebDriver/Chrome/Options.html
#https://groups.google.com/g/robotframework-users/c/r3rxgSv1Cs0
# https://www.selenium.dev/documentation/webdriver/getting_started/first_script/    instruction

# Testing
# options = Options()
# language = "en-US"
# options.add_argument(f"--lang={language}")

# driver = webdriver.Chrome(options=options)
# driver.maximize_window()
# driver.get(game_url)



In [74]:
# driver.quit()

## Handy web scrape functions: 



https://www.geeksforgeeks.org/execute_script-driver-method-selenium-python/
https://developer.mozilla.org/en-US/docs/Web/API/Window

Most Browser have a windows as a alias to the tab

In [75]:
def get_Y_scroll_position(driver):
    return driver.execute_script("return window.pageYOffset")


# https://stackoverflow.com/questions/42982950/how-to-scroll-down-the-page-till-bottomend-page-in-the-selenium-webdriver

def scroll_bottom(driver):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    time.sleep(random.uniform(1, 3))

    


# what do I want?
# Recommended or not/hours played/actual review/ early acess or not
def get_review_data(card, game_id, name):     
    # Retrieve the Steam ID (or "Private ID" if not a valid digit)
    steam_id = get_steam_id(card)
    
    # Retrieve the number of hours played
    game_time = get_hour(card)
    
    # Retrieve the recommendation status (e.g., Recommended or Not)
    recommend = get_recommendation(card)
    
    # Retrieve the helpful review count
    helpful_count = get_helpful(card)
    
    # Retrieve the date the review was posted
    review_date = get_date(card)
    
    # Retrieve the comment or review text
    review_comment = get_comment(card)
    
    # Retrieve whether the game is early access or fully released
    early_access = get_early_access(card)
    
    refund = get_refund(card)
    
    # Return all the collected data in a dictionary or tuple
    return {
        "steam_id": steam_id,
        "game_time": game_time,
        "recommend": recommend,
        "helpful_count": helpful_count,
        "review_date": review_date,
        "review_comment": review_comment,
        "early_access": early_access,
        "refund": refund,
        "game_id": game_id,
        "name" : name
        
    }
    

# https://www.browserstack.com/guide/find-element-by-xpath-in-selenium
# https://selenium-python.readthedocs.io/locating-elements.html
 # the elments are index by 1 start
from selenium.common.exceptions import NoSuchElementException

def get_steam_id(card):
    try:
        profile = card.find_element(By.XPATH, './/div[@class="apphub_friend_block"]/div/a[2]').get_attribute('href')
        id = profile.split("/")[-2]
        if id.isdigit():
            return int(id)
        else:
            return "Private ID" # Return empty string if the id is not a digit
    except NoSuchElementException:
        return "Not Found"  # Return empty string if element is not found

def get_hour(card):
    try:
        return card.find_element(By.CLASS_NAME, "hours").text
    except NoSuchElementException:
        return "Not Found"  # Return empty string if element is not found

def get_recommendation(card):
    try:
        return card.find_element(By.CLASS_NAME, "title").text
    except NoSuchElementException:
        return "Not Found"  # Return empty string if element is not found

def get_helpful(card):
    try:
        return card.find_element(By.CLASS_NAME, "found_helpful").text.split("\n")[0]
    except NoSuchElementException:
        return "Not Found"  # Return empty string if element is not found

def get_date(card):
    try:
        return card.find_element(By.CLASS_NAME, "date_posted").text
    except NoSuchElementException:
        return "Not Found"  # Return empty string if element is not found

def get_comment(card):
    try:
        text = card.find_element(By.XPATH, './/div[@class ="apphub_CardTextContent"]').text
        lines = text.split("\n")[-1]  # Assuming you want the last line
        return lines
    except NoSuchElementException:
        return "Not Found"  # Return empty string if element is not found

def get_early_access(card):
    try:
        early_access = card.find_element(By.CLASS_NAME, "early_access_review").text
        return early_access
    except NoSuchElementException:
        return "Full_Release"  # Default value if element is not found
    
def get_refund(card):
    try:
        early_access = card.find_element(By.CLASS_NAME, "refunded").text
        return early_access
    except NoSuchElementException:
        return "not refunded"  # Default value if element is not found





# Test for actually getting review card
# review_cards = driver.find_elements(By.CLASS_NAME, "apphub_Card")  # Returns a list

# print(review_cards)

# if len(review_cards) > 2:  # Ensure there are enough elements
#     print(get_steam_id(review_cards[7]))  # Pass a single element
# else:
#     print("Not enough review cards found.")
    
# print(get_hour(review_cards[4]))
# print(get_helpful(review_cards[1]))
# print(get_date(review_cards[1]))
# print(get_comment(review_cards[5]))
# print(get_early_access(review_cards[9]))

# print(driver.find_elements(By.CLASS_NAME, "hours")[1].text) #change web element into text
    

In [ ]:
def get_Y_scroll_position(driver):
    return driver.execute_script("return window.pageYOffset")


# https://stackoverflow.com/questions/42982950/how-to-scroll-down-the-page-till-bottomend-page-in-the-selenium-webdriver

def scroll_bottom(driver):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    time.sleep(1)
    


# single game scrapping 
reviews = []
id_set = set()
max_scroll = 40


def scrap_steam_data(driver,game_id,name):
    try:
        previous_position = get_Y_scroll_position(driver)
        state = True
        scroll_count = 0
        while(state):
            element = driver.find_elements(By.CLASS_NAME, "apphub_Card")
            
            for card in element:
                steam_id = get_steam_id(card)         
                if steam_id in id_set:
                    continue
                else:
                    review_package = get_review_data(card, game_id,name)
                    reviews.append(review_package)
                    id_set.add(steam_id)
                    
                
            while(scroll_count < max_scroll):
                scroll_bottom(driver)
                current_position = get_Y_scroll_position(driver)                

                
                # This means we reached the end of the scrolling page
                if(previous_position == current_position):
                    scroll_count+=1
                    # safer random break less likly to get blocked
                    
                    time.sleep(random.uniform(1, 3))
                    
                    # give the program a bit slack maybe its the loading time issue
                    if(scroll_count>max_scroll/2):
                       state = False
                       break
                else:
                    previous_position = current_position
                    break
    except Exception as e:
        # Handles the exception
        print(f"An error occurred: {e}")
                              
        
            
            
             
            # limit one comment per id, this is interesting cause some of the user would turn privacy on and the id is actually blank so beware 
            # perhaps not such a big deal since all such privacy on users would have the string "id" as their id hence appering once as an edge case
    
    

     

## Actually scraping


In [77]:


# https://stackoverflow.com/questions/30729318/print-into-console-terminal-not-into-cell-output-of-ipython-notebook



reviews = []
id_set = set()
max_scroll = 40
print("hello")


good_review_id = [413150,620,2379780,1145360,2527500,294100,524220,2183900]
good_review_name = ["Stardew Valley","Portal 2","Balatro","MiSide","RimWorld","NieRAutomata","Warhammer 40,000: Space Marine 2"]
mix_review_id = [2246340,1295660,1172470,1240440,1124300,2878980,216150]
mix_review_name = ["Monster_Hunter_Wilds","Sid Meier's Civilization VII","Apex Legends","Halo_Infinite","HUMANKIND","NBA 2K25","MapleStory"]
bad_review_id = [1180320,1419640,2357570,1367080,2537590]
bad_review_name = ["War_of_the_Three_Kingdoms","Project F","Overwatch® 2","MOBILE SUIT GUNDAM BATTLE OPERATION 2","Microsoft Flight Simulator 2024"]

options = Options()
language = "en-US"
options.add_argument(f"--lang={language}")
driver = webdriver.Chrome(options=options)
driver.maximize_window()

game_ids = good_review_id + mix_review_id + bad_review_id
game_names = good_review_name + mix_review_name + bad_review_name

# Generate Steam review URLs
for game_id, game_name in zip(game_ids, game_names):
    game_url = f"https://steamcommunity.com/app/{game_id}/reviews/?p=1&browsefilter=trendyear&filterLanguage=english"
    print(f"{game_name}: {game_url}")
    driver.get(game_url)
    scrap_steam_data(driver, game_id, game_name)
    
    
if reviews:
    # Convert the list of dictionaries (reviews) into a pandas DataFrame
    reviews_df = pd.DataFrame(reviews)
    
    # Save the DataFrame to a CSV file
    reviews_df.to_csv("reviews.csv", index=False)
    print("Reviews have been successfully saved to 'reviews.csv'.")
else:
    print("No reviews were collected.")
    
    


driver.quit()


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=134.0.6998.89)
Stacktrace:
0   chromedriver                        0x000000010331d798 cxxbridge1$str$ptr + 2785964
1   chromedriver                        0x0000000103315d70 cxxbridge1$str$ptr + 2754692
2   chromedriver                        0x0000000102e69ea8 cxxbridge1$string$len + 92928
3   chromedriver                        0x0000000102e43a78 chromedriver + 129656
4   chromedriver                        0x0000000102ed9498 cxxbridge1$string$len + 549104
5   chromedriver                        0x0000000102ef2178 cxxbridge1$string$len + 650704
6   chromedriver                        0x0000000102ea535c cxxbridge1$string$len + 335796
7   chromedriver                        0x00000001032e2c68 cxxbridge1$str$ptr + 2545532
8   chromedriver                        0x00000001032e5f34 cxxbridge1$str$ptr + 2558536
9   chromedriver                        0

## Data cleaning


## data loading


In [ ]:
driver.quit()

# Handy Function for player count

We might need an dynamically increase